# LastMile Instrumentor for LangChain


In this notebook, we showcase how to use the **LastMile Tracing SDK** to auto-instrument tracing for a LangChain "Plan-and-Execute" agent. With tracing automatically setup, you can easily debug your application using LastMile's RAG Debugger.

The LangChain "Plan-and-Execute" agent accomplishes an objective by first planning what to do and then executing the sub-tasks. LastMile autoinstrumentation is used to trace and monitor the execution of the agent.

## Notebook Outline
* [Step 1: Setup](#setup)
* [Step 2: Configure the LastMile Instrumentor](#step2)
* [Step 3: Define Tools for LangChain Agent](#step3)
* [Step 4: Run example with LangChain Agent](#step4)
* [Step 5: View Trace Data in RAG Debugger](#step5)

<a name="setup"></a>
# Step 1: Setup

To begin, we need to install a few packages including langchain and lastmile-eval.

In [1]:
# !pip install langchain
# !pip install langchain_community
# !pip install langchain_core
# !pip install langchain_experimental
# !pip install langchain_openai
# !pip install wikipedia
# !pip install numexpr
# !pip install lastmile_eval
# !pip install "tracing-auto-instrumentation[langchain]" --upgrade

Import the necessary libraries for this example.

In [2]:
from langchain.chains import LLMMathChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

Before we start this tutorial, we need the following tokens/keys:

* LastMile AI API Token: Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* OpenAI API Key: Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

You can either set these explicitly (uncomment the lines below), or save them in a `.env` file within this project directory.


In [3]:
import dotenv
dotenv.load_dotenv()

# import os
# os.environ['OPENAI_API_KEY'] =  os.getenv('OPENAI_API_KEY')
# os.environ['LASTMILE_API_TOKEN'] =  os.getenv('LASTMILE_API_TOKEN')

True

<a name="step2"></a>

# Step 2: Configure the LastMile Instrumentor

Next, we create an instance of `LangChainInstrumentor` with a project name. The `instrument()` method is called to instrument the code for tracing and monitoring.

In [4]:
from tracing_auto_instrumentation.langchain import LangChainInstrumentor

# Create an instance of LangChainInstrumentor and instrument the code
instrumentor = LangChainInstrumentor(project_name="Plan-and-Execute Example")
instrumentor.instrument()

[DEBUG] 2024-06-13 16:31:57,821 connectionpool.py:1055: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-06-13 16:31:58,030 connectionpool.py:549: https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=Plan-and-Execute+Example HTTP/1.1" 200 359


<a name="step3"></a>

# Step 3: Define Tools for Langchain Agent

The tools that the agent will use for executing sub-tasks are defined. In this example, we have two tools:
- "Wikipedia" tool: Uses the `WikipediaAPIWrapper` to look up information on Wikipedia.
- "Calculator" tool: Uses the `LLMMathChain` to perform mathematical calculations.

In [5]:
# Define the tools that the agent will use for executing sub-tasks
wikipedia = WikipediaAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to look up information on Wikipedia",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to perform mathematical calculations",
    ),
]


[DEBUG] 2024-06-13 16:31:58,147 _config.py:80: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] 2024-06-13 16:31:58,148 _config.py:146: load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'
[DEBUG] 2024-06-13 16:31:58,154 _config.py:80: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] 2024-06-13 16:31:58,155 _config.py:146: load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'


<a name="step4"></a>

# Step 4: Run example with LangChain Agent

Let's try an example with our LangChain agent. We will ask a simple question "What is hte population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."

The agent needs to look up the population (using Wikipedia) and then also make a calculation.

In [6]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

# ## Run Example

# Run the agent with a sample question
question = "What is the population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."
result = agent.run(question)

[DEBUG] 2024-06-13 16:31:58,251 _config.py:80: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] 2024-06-13 16:31:58,252 _config.py:146: load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'
[DEBUG] 2024-06-13 16:31:58,258 _config.py:80: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] 2024-06-13 16:31:58,258 _config.py:146: load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'
/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
[DEBUG] 2024-06-13 16:31:58,279 _base_client.py:446: Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messa



> Entering new AgentExecutor chain...


[DEBUG] 2024-06-13 16:32:01,828 _trace.py:45: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 13 Jun 2024 20:32:01 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'1340'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999667'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'9ms'), (b'x-request-id', b'req_ad491d22cbcf6c42047686b67039dc2c'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8934d8f1bfe7197c-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
[INFO] 2024-06-13 16:32:01,829 _client.py:1026: HTT

Thought: The Eiffel Tower is located in Paris, France. I can provide the answer directly without the need for any tools.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The Eiffel Tower is located in France."
}
```  


> Finished chain.


> Entering new AgentExecutor chain...


[DEBUG] 2024-06-13 16:32:02,835 _trace.py:45: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 13 Jun 2024 20:32:02 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'735'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999636'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'10ms'), (b'x-request-id', b'req_3222b9835362f624c386a3af838d9a76'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8934d8fc4aa7197c-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
[INFO] 2024-06-13 16:32:02,836 _client.py:1026: HTT

...

[DEBUG] 2024-06-13 16:32:13,894 _trace.py:45: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 13 Jun 2024 20:32:13 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'1382'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999477'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'15ms'), (b'x-request-id', b'req_df774a8f25de4fa297ae0c84f9bf8c7a'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8934d93d78ac197c-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
[INFO] 2024-06-13 16:32:13,895 _client.py:1026: HT

I have calculated the square root of the population of Paris, which is approximately 2,102,650 residents, to be 1450.0517232154168.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The square root of the population of Paris is approximately 1450.05."
}
```

> Finished chain.


> Entering new AgentExecutor chain...


[DEBUG] 2024-06-13 16:32:15,096 _trace.py:45: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 13 Jun 2024 20:32:15 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'852'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999502'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'14ms'), (b'x-request-id', b'req_29b4a9483d42b1bfeb7369d1b0cb8d29'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8934d9479a78197c-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
[INFO] 2024-06-13 16:32:15,096 _client.py:1026: HTT

Thought: The final answer is already calculated in the previous steps, so I can provide it directly to the user.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The square root of the population of Paris is approximately 1450.05."
}
```

> Finished chain.


[DEBUG] 2024-06-13 16:32:15,394 connectionpool.py:549: https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
[DEBUG] 2024-06-13 16:32:15,396 connectionpool.py:1055: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-06-13 16:32:15,486 connectionpool.py:549: https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
[DEBUG] 2024-06-13 16:32:15,488 connectionpool.py:1055: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-06-13 16:32:15,576 connectionpool.py:549: https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
[DEBUG] 2024-06-13 16:32:15,578 connectionpool.py:1055: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-06-13 16:32:15,660 connectionpool.py:549: https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 None
[DEBUG] 2024-06-13 16:32:15,662 connectionpool.py:1055: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-06-13 16:32:15,743 connecti

In [7]:
print(result)

The square root of the population of Paris is approximately 1450.05.


<a name="step5"></a>

# Step 5: View Trace Data in RAG Debugger
Now we can view the trace data of our LangChain agent in a UI!
#### From your terminal:

Export your LASTMILE_API_TOKEN

```bash
export LASTMILE_API_TOKEN="<your-api-token>"
```

Run this CLI command to access the UI

```bash
rag-debug launch
```
Navigate to the 'Traces' Page where you see all the Traces listed under this Project (top-right corner to choose Project).

<img width="973" alt="Screenshot 2024-05-28 at 12 06 53 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/a2ad1630-dc62-43a6-8d81-cfa335fe8fb1"/>


Let's click into the Trace.

<img width="973" alt="Screenshot 2024-05-28 at 12 07 04 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/01bb1693-a79c-40a6-962e-15446cbe03c6"/>


Here we can see all the spans auto-generated for us. This can help us debug and pinpoint issues in our application, especially if we add additional logging on top of the auto-instrumentor.